# EfficientNetB0


## I. Setup


In [1]:
import os
import sys

# os.chdir('/Volumes/SD Card/cleaned')
# os.getcwd()

sys.path.append('..')

In [2]:
def listdir(path: str):
    return os.listdir(path)

In [3]:
dataset_path = '/Volumes/SD Card/cleaned'

# cosonant_path = f'{dataset_path}/cosonant'
# independent_vowel_path = f'{dataset_path}/independent_vowel'
# stack_consonant_vowel_path = f'{dataset_path}/stack_consonant'
# sub_vowel_path = f'{dataset_path}/sub_vowel'
# vowel_path = f'{dataset_path}/vowel'

# images_count = 0
# for inner1 in listdir(f'{dataset_path}'):
#     for inner2 in listdir(f'{dataset_path}/{inner1}'):
#         try:
#             images_count = images_count + len(listdir(f'{dataset_path}/{inner1}/{inner2}'))
#         except NotADirectoryError as e:
#             # print(f'ERROR: {e}')
#             continue

# images_count

In [4]:
# only test consonant for now
cosonant_path = '/Volumes/SD Card/cleaned_hand'

## II. Prepare dataset


In [5]:
import pandas as pd
import random


In [6]:
df = pd.DataFrame(columns=["label", "path"])

In [7]:
for cosonant in listdir(cosonant_path):
    variants_path = f'{cosonant_path}/{cosonant}'
    variants = listdir(f'{cosonant_path}/{cosonant}')

    for variant in variants:
    # for variant in random.sample(variants, min(10, len(variants))):
    # for variant in [variants[0], variants[1], variants[2], variants[3]]:
        vpath = f'{variants_path}/{variant}'
        df.loc[len(df.index)] = [cosonant, vpath]

In [8]:
df.shape

(7737, 2)

In [9]:
df.head(5)

,label,path
0,vo,/Volumes/SD Card/cleaned_hand/vo/162.jpg
1,vo,/Volumes/SD Card/cleaned_hand/vo/176.jpg
2,vo,/Volumes/SD Card/cleaned_hand/vo/177.jpg
3,vo,/Volumes/SD Card/cleaned_hand/vo/163.jpg
4,vo,/Volumes/SD Card/cleaned_hand/vo/48.jpg


In [10]:
df["label"].unique()


array(['vo', 'cha', 'cho', 'chha', 'tho1', 'chho', 'mo', 'da', 'do', 'no',
       'na', 'nho', 'tha1', 'ro', 'yo', 'sa', 'po', 'kha', 'kho', 'ba',
       'ha', 'tho2', 'ka', 'ko', 'ngo', 'or', 'tha2', 'lo', 'la', 'ta',
       'to', 'pho', 'pha'], dtype=object)

## III. Preprocessing


In [11]:
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)

### a. Image


In [12]:
import tensorflow as tf
import cv2
import numpy as np

images = []
labels = []
error_count = 0


def resize_image(image_path):
    image = cv2.imread(image_path)
    image = tf.image.resize(image, size=(IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, dtype=tf.float32)
    return image


for item in np.array(df):
    label = item[0]
    path = item[1]

    try:
        image = resize_image(path)
        images.append(image)
        labels.append(label)
    except ValueError as e:
        error_count = error_count + 1

error_count, len(images), len(labels)


2022-11-18 16:12:20.727002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 16:12:23.835037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(0, 7737, 7737)

In [13]:
images = np.array(images)
labels = np.array(labels)


### b. Label


In [14]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(labels)
Y = y.reshape(-1, 1)

Y.shape


(7737, 1)

### c. Split testing & training data


In [15]:
from sklearn.model_selection import train_test_split


In [16]:
train_x, test_x, train_Y, test_Y = train_test_split(
    images,
    Y,
    test_size=0.2,
    random_state=10
)


In [17]:
train_x.shape, test_x.shape, train_Y.shape, test_Y.shape


((6189, 224, 224, 3), (1548, 224, 224, 3), (6189, 1), (1548, 1))

## IV. Train


In [18]:
NUM_CLASSES = len(labels)

In [19]:
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    classes=NUM_CLASSES
)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["accuracy"]
)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7737)             13960668  
                                                                 
Total params: 13,960,668
Trainable params: 13,918,645
Non-trainable params: 42,023
_________________________________________________________________


In [20]:
from datetime import datetime

In [21]:
epochs = 3

started_at = datetime.now().time()
hist = model.fit(train_x, train_Y, epochs=epochs, verbose=2)
trained_at = datetime.now().time()

hist.history

Epoch 1/3
194/194 - 1139s - loss: 310.1240 - accuracy: 1.6158e-04 - 1139s/epoch - 6s/step
Epoch 2/3
194/194 - 1024s - loss: 310.1241 - accuracy: 0.0000e+00 - 1024s/epoch - 5s/step
Epoch 3/3
194/194 - 931s - loss: 310.1241 - accuracy: 3.2315e-04 - 931s/epoch - 5s/step


{'loss': [310.1240234375, 310.1240539550781, 310.12408447265625],
 'accuracy': [0.00016157698701135814, 0.0, 0.0003231539740227163]}

In [22]:
started_at, trained_at

(datetime.time(16, 14, 45, 519306), datetime.time(17, 6, 20, 805646))

In [23]:
prediction = model.evaluate(test_x, test_Y)

print("Loss = " + str(prediction[0]))
print("Test Accuracy = " + str(prediction[1]))

49/49 [==============================] - 55s 1s/step - loss: 319.9721 - accuracy: 0.0000e+00
Loss = 319.97210693359375
Test Accuracy = 0.0


## V. Save Model

In [24]:
model.save(f'../models/efficient_net_b0-{trained_at}.h5')